# Summary
This code will compare stock market info from the US and China to the price of bitcoin and to the "impressions" of bitcoin and cryptocurrency on Google trends. For China, the Shanghai Stock Exchange (SSE) Composite is used as it is the most comprabable to the NASDAQ. Both of these markets indicate the general "health" of the respective country's economy. Regression techniques will be run on the data in order to predict the price of bitcoin on the following day. A webapp http://mohibtc.pythonanywhere.com/ will ask the user for what price they bought their bitcoin at. If the model predicts tomorrow's price to be less than what the user bought bitcoin for, it will instruct them to sell. If the model predict's tomorrow's price to be more than what the user bought bitcoin for, it will instruct them to buy more. Multiple regressions will be used to determine the best model.

In [1]:
# read data into a DataFrame and importing all regression related libraries.
import pandas as pd
import pylab as plt
import seaborn
from sklearn.linear_model import LinearRegression
import numpy.random as nprnd
import random
import json
import statsmodels.api as sm
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import r2_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import re

C:\Users\tuleg\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\tuleg\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
#closing price of BTC on this exchange is 23:59 PM UTC which is 7:59 EST; opening at 12 UTC
#NYSE closes at  is 21:00 UTC which is 4PM EST; opens at 14:30 UTC
#Shanghai stock exchange closes at 7:00 UTC which is 3AM EST; opens at 1:30 UTC
#To compensate, Shanghai stock exchange data will be staggered as it is 12 hours ahead of US. 

#taken from https://www.coindesk.com/price/
dfBTC=pd.read_csv("/Users/tuleg/Documents/python-introduction-MohiAhmed/coindesk-bpi-USD-close_data-2017-05-07_2018-05-07.csv")
dfBTC.head()
#matrix needs to be flipped so that row 0 corresponds to today's closing price

,Date,Close Price
0,2017-05-07 00:00:00,1555.47
1,2017-05-08 00:00:00,1639.32
2,2017-05-09 00:00:00,1706.93
3,2017-05-10 00:00:00,1756.80
4,2017-05-11 00:00:00,1807.37


In [8]:
#taken from https://finance.yahoo.com/quote/000001.SS/history/
dfSSE=pd.read_csv("/Users/tuleg/Documents/python-introduction-MohiAhmed/SSE.csv")
dfSSE.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-04-09,3125.441895,3146.093018,3110.302979,3138.293945,3138.293945,139600
1,2018-04-10,3144.257080,3190.648926,3139.081055,3190.322021,3190.322021,168200
2,2018-04-11,3197.372070,3220.844971,3191.586914,3208.082031,3208.082031,175900
3,2018-04-12,3203.278076,3205.251953,3177.049072,3180.157959,3180.157959,148200
4,2018-04-13,3192.041992,3197.895996,3155.506104,3159.052002,3159.052002,127600


In [9]:
dfNAS=pd.read_csv("/Users/tuleg/Documents/python-introduction-MohiAhmed/NASDAQ.csv")
dfNAS.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-04-09,6971.450195,7074.950195,6944.959961,6950.339844,6950.339844,2080900000
1,2018-04-10,7060.990234,7117.979980,7014.879883,7094.299805,7094.299805,2238770000
2,2018-04-11,7055.000000,7128.540039,7055.000000,7069.029785,7069.029785,1852120000
3,2018-04-12,7112.020020,7166.000000,7105.089844,7140.250000,7140.250000,2021110000
4,2018-04-13,7179.620117,7183.620117,7078.140137,7106.649902,7106.649902,1743640000
